In [26]:
import pandas as pd
import os

In [27]:
csv_files = [f for f in os.listdir('files/olds')]
for file in csv_files:
    df = pd.read_csv(f'files/olds/{file}')
    columns = ['Data', 'Hora UTC', 'PRECIPITA TOTAL, HORARIO (mm)' ]
    df = df[columns]
    df = df.drop(columns=['Hora UTC'])
    df['PRECIPITA TOTAL, HORARIO (mm)'] = df['PRECIPITA TOTAL, HORARIO (mm)'].str.replace(',', '.')
    df['PRECIPITA TOTAL, HORARIO (mm)'] = pd.to_numeric(df['PRECIPITA TOTAL, HORARIO (mm)'], errors='coerce')
    df.loc[df["PRECIPITA TOTAL, HORARIO (mm)"] < 0, "PRECIPITA TOTAL, HORARIO (mm)"] = None

    while df['PRECIPITA TOTAL, HORARIO (mm)'].isna().sum() > 0:
        df['PRECIPITA TOTAL, HORARIO (mm)'] = df['PRECIPITA TOTAL, HORARIO (mm)'].fillna(method='ffill').fillna(method='bfill')
    df_grouped = df.groupby("Data").sum().reset_index()
    file_name = file.replace('.CSV', '.csv')
    df_grouped.to_csv(f'files/{file_name}', index=False)
    


/tmp/ipykernel_8358/401730537.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['PRECIPITA TOTAL, HORARIO (mm)'] = df['PRECIPITA TOTAL, HORARIO (mm)'].fillna(method='ffill').fillna(method='bfill')


KeyError: "['Hora UTC'] not in index"

In [3]:
# failed = pd.read_csv('files/olds/INMET_SE_SP_A755_BARUERI_01-01-2016_A_31-12-2016.CSV')[columns]
# # failed = df.drop(columns=['Hora UTC'])
# failed['PRECIPITA TOTAL, HORARIO (mm)'] = failed['PRECIPITA TOTAL, HORARIO (mm)'].str.replace(',', '.')
# failed['PRECIPITA TOTAL, HORARIO (mm)'] = pd.to_numeric(failed['PRECIPITA TOTAL, HORARIO (mm)'])
# failed.loc[failed["PRECIPITA TOTAL, HORARIO (mm)"] < 0, "PRECIPITA TOTAL, HORARIO (mm)"] = None

# failed.to_csv('teste.csv', index=False)

In [6]:
csv_files = [f for f in os.listdir('files') if os.path.isfile(os.path.join('files', f))]
for file in csv_files:
    df = pd.read_csv(f'files/{file}')
    df['Data'] = df['Data'].str.replace('/', '-')
    df['Data'] = pd.to_datetime(df['Data'])
    df.to_csv(f'files/{file}', index=False)


KeyError: 'Data'

In [9]:
combined_df = pd.concat([pd.read_csv(f'files/{file}') for file in csv_files if file != 'combined_data.csv'])
combined_df.to_csv('combined_data.csv', index=False)

In [12]:
df = pd.read_csv('combined_data.csv')
df["Data"] = pd.to_datetime(df["Data"])

additional_data = {
    "Data": ["2011-03-27", "2011-03-28"],
    "PRECIPITA TOTAL, HORARIO (mm)": [0.0, 0.0]
}
additional_df = pd.DataFrame(additional_data)
additional_df["Data"] = pd.to_datetime(additional_df["Data"])
df = pd.concat([additional_df, df]).reset_index(drop=True)
df["Data"] = pd.to_datetime(df["Data"])

nat_rows = df[df["Data"].isna()]

df["Week"] = df["Data"].dt.to_period('W-SAT').apply(lambda r: r.start_time)

df_grouped = df.drop(columns=['Data']).groupby("Week").sum().reset_index()

df_grouped.head(10)

         Data  PRECIPITA TOTAL, HORARIO (mm)
0  2011-03-29                            0.0
1  2011-03-30                            0.0
2  2011-03-31                            0.0
3  2011-04-01                            0.0
4  2011-04-02                            0.6
0    2011-03-29
1    2011-03-30
2    2011-03-31
Name: Data, dtype: object


,Week,"PRECIPITA TOTAL, HORARIO (mm)"
0,2011-03-27,1.2
1,2011-04-03,44.8
2,2011-04-10,88.4
3,2011-04-17,22.4
4,2011-04-24,12.4
5,2011-05-01,14.4
6,2011-05-08,2.0
7,2011-05-15,30.0
8,2011-05-22,0.4
9,2011-05-29,1.2


In [25]:
df_grouped.to_csv('files/combined_data.csv', index=False)

In [20]:
# df_grouped.\
#     iloc[::-1].\
#     reset_index(drop=True).\
#     to_csv('files/combined_data.csv', index=False)